# P2. Les vues SQL

## Définition d'une vue SQL  

Une vue n'est rien de plus qu'une instruction SQLite qui est stockée dans la base de données avec un nom associé. Il s'agit en fait d'une composition d'une table sous la forme d'une requête SQLite prédéfinie.

Une vue peut contenir toutes les lignes d'une table ou des lignes sélectionnées d'une ou plusieurs tables. Une vue peut être créée à partir d'une ou plusieurs tables qui dépendent de la requête SQLite écrite pour créer une vue.

## Code utilitaire

In [1]:
import os
import sqlite3
import pandas as pd   

In [2]:
from pathlib import Path

In [3]:
# Ouvre connection vers SQLite db
chemin_bdd = Path('db') / 'chinook.db'
conn = sqlite3.connect(chemin_bdd)

In [4]:
# Fonction permettant d'exécuter un requête SQL sur une BDD définie par sa connexion conn
def executer_requete(requete_sql, conn):
    try:
        cursor = conn.cursor()
        cursor.execute(requete_sql)
        conn.commit()
    except sqlite3.Error as e:
        print("Erreur lors de l'execution de la requête")
        print(e)
        return
    cursor.close()


## Example de vue SQL: v_albums

La requête suivante permet d'obtenir une liste (simplifiée) des albums / artistes correspondants:  

In [5]:
# Requete sur les tables albums & artists
requete = """
    SELECT AlbumId,
           Title,
           a.Name AS Artist
    FROM albums
    JOIN artists a on albums.ArtistId = a.ArtistId
    ORDER BY Artist;
"""

albums = pd.read_sql_query(requete, conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


Le code ci-dessous permet de créer une vue correspondant à la requête ci-dessous :    
Référence vue sous SQLite : https://www.sqlite.org/lang_createview.html

In [6]:
# creation vue albums avec artistes
creation_vue_albums = """
    CREATE VIEW v_albums (
        AlbumId,
        Title,
        Artist)
    AS
        SELECT AlbumId,
               Title,
               a.Name AS Artist
        FROM albums
        JOIN artists a on albums.ArtistId = a.ArtistId
        ORDER BY Artist, Title;
"""

executer_requete(creation_vue_albums, conn)

Une fois la vue créé, on peut la réutiliser dans des requêtes SQL comme une table:  

In [7]:
# Utilisation de la vue v_albums
albums = pd.read_sql_query("SELECT * FROM v_albums LIMIT 10;", conn)
albums.head(5)

,AlbumId,Title,Artist
0,1,For Those About To Rock We Salute You,AC/DC
1,4,Let There Be Rock,AC/DC
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra
3,267,Worlds,Aaron Goldberg
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...


In [8]:
# Utilisation de la vue v_albums avec une clause WHERE
albums = pd.read_sql_query("SELECT * FROM v_albums WHERE Artist = 'U2';", conn)
albums

,AlbumId,Title,Artist
0,232,Achtung Baby,U2
1,233,All That You Can't Leave Behind,U2
2,234,B-Sides 1980-1990,U2
3,235,How To Dismantle An Atomic Bomb,U2
4,255,Instant Karma: The Amnesty International Campa...,U2
5,236,Pop,U2
6,237,Rattle And Hum,U2
7,238,The Best Of 1980-1990,U2
8,239,War,U2
9,240,Zooropa,U2


In [9]:
# Utilisation de la vue v_albums avec une autre jointure
requete = """
    SELECT a.AlbumId,
           a.Title,
           a.Artist,
           count(t.TrackId) as Tracks
    FROM v_albums a
    JOIN tracks t ON a.AlbumId = t.AlbumId
    GROUP BY 1, 2, 3
    ORDER BY 3, 2;
"""
albums = pd.read_sql_query(requete, conn)
albums.head()

,AlbumId,Title,Artist,Tracks
0,1,For Those About To Rock We Salute You,AC/DC,10
1,4,Let There Be Rock,AC/DC,8
2,296,"A Copland Celebration, Vol. I",Aaron Copland & London Symphony Orchestra,1
3,267,Worlds,Aaron Goldberg,1
4,280,The World of Classical Favourites,Academy of St. Martin in the Fields & Sir Nevi...,2


In [10]:
# Effacer une vue
executer_requete("DROP VIEW IF EXISTS v_albums;", conn)

### Question: Avantages & Inconvénients des vues SQL

Quelles sont les avantages et inconvénients des vues SQL ?  
avantage: 
1. GAGNER DU TEMPS SUR L’ÉCRITURE DE REQUÊTES
2. CENTRALISATION DES RÈGLES MÉTIER 
3. SÉCURISATION DE L’ACCÈS AUX DONNÉES(VUES AVEC DROITS PARAMÉTRÉS)
inconvénient: 
L'un des principaux inconvénients de l'utilisation de la vue apparaît lorsque nous modifions fréquemment les structures de table sur lesquelles la vue est créée. Ainsi, lorsque les structures de table sont modifiées, la vue doit également être modifiée.


---

## Exercices de création / mise en oeuvre de vues

### Vue 1: caractéristiques complètes des Albums  
A partir de l'exemple de vue ci-dessus, créer et exploiter une nouvelle vue v_albums reprenant pour chaque album:  
- AlbumId
- Title
- Artist
- nb de pistes
- Durée totale de l'album (en minutes)
- Prix de l'album

In [11]:
executer_requete("DROP VIEW IF EXISTS V_albums;", conn)
creation_vue_albums = """
 CREATE VIEW V_albums (
        AlbumId,
        Title,
        Artist,
        NBpiste,
        Durée_min,
        prix)
    AS
        SELECT albums.AlbumId,
               albums.Title,
               artists.Name AS Artist,
               COUNT(tracks.TrackId) AS NBpiste,
               SUM(tracks.Milliseconds)/60000 AS Durée_min,
               SUM(tracks.UnitPrice) AS prix   
        FROM tracks
        JOIN albums ON tracks.AlbumId=albums.AlbumId 
        JOIN artists ON albums.ArtistId =artists.ArtistId
        GROUP BY albums.AlbumId
"""

executer_requete(creation_vue_albums, conn)

In [12]:
albums = pd.read_sql_query("SELECT * FROM V_albums;", conn)
albums.head(5)

,AlbumId,Title,Artist,NBpiste,Durée_min,prix
0,1,For Those About To Rock We Salute You,AC/DC,10,40,9.90
1,2,Balls to the Wall,Accept,1,5,0.99
2,3,Restless and Wild,Accept,3,14,2.97
3,4,Let There Be Rock,AC/DC,8,40,7.92
4,5,Big Ones,Aerosmith,15,73,14.85


### Vue 2: caractéristiques détaillées des pistes
Créer et exploiter une nouvelle vue v_tracks reprenant pour chaque piste:  
- TrackId
- Name
- Genre
- Album
- Artist
- Composer
- Temps (en seconde)
- MediaType

In [13]:
executer_requete("DROP VIEW IF EXISTS v_track;", conn)

creation_vue_albums = """CREATE VIEW v_track (
        trackId,
        Title,
        Genret,
        Album,
        Artist,
        Composer,
        Temps_sec,
        MediaType)
    AS
        SELECT tracks.trackId AS trackId,
               tracks.Name AS Title,
               genres.Name AS Genre,
               albums.Title AS Album,
               artists.Name AS Artist,
               tracks.Composer AS Composer,
               tracks.Milliseconds/1000 AS Temps_sec,
               Media_types.Name AS MediaType
        FROM tracks
        JOIN albums ON tracks.AlbumId=albums.AlbumId 
        JOIN artists ON albums.ArtistId = artists.ArtistId
        JOIN Genres ON tracks.GenreId=genres.GenreId 
        JOIN media_types ON tracks.MediaTypeId=media_types.MediaTypeId
        GROUP BY tracks.trackId
        """

In [14]:
executer_requete(creation_vue_albums, conn)

In [15]:
tracks = pd.read_sql_query("SELECT * FROM v_track LIMIT 10;", conn)
tracks.head(5)

,trackId,Title,Genret,Album,Artist,Composer,Temps_sec,MediaType
0,1,For Those About To Rock (We Salute You),Rock,For Those About To Rock We Salute You,AC/DC,"Angus Young, Malcolm Young, Brian Johnson",343,MPEG audio file
1,2,Balls to the Wall,Rock,Balls to the Wall,Accept,None,342,Protected AAC audio file
2,3,Fast As a Shark,Rock,Restless and Wild,Accept,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230,Protected AAC audio file
3,4,Restless and Wild,Rock,Restless and Wild,Accept,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252,Protected AAC audio file
4,5,Princess of the Dawn,Rock,Restless and Wild,Accept,Deaffy & R.A. Smith-Diesel,375,Protected AAC audio file


### Vue 3: Vue augmentée invoices
A. Créer une nouvelle vue v_invoices à partir de la table invoices ajoutant 3 nouveaux attributs basés sur l'attribut `InvoiceDate`:  
- semaine de la transaction
- mois de la transaction
- année de la transaction

In [16]:
executer_requete("DROP VIEW IF EXISTS v_invoices;", conn)
creation_v_invoices = """
    CREATE VIEW v_invoices (
        InvoiceId,
        CustomerId,
        InvoiceDate,
        InvoiceSemaine,
        InvoiceMois,
        InvoiceAnnée,
        BillingAddress,
        BillingCity,
        BillingState,
        BillingCountry,
        BillingPostalCode,
        Total)
    AS
        SELECT InvoiceId,
            CustomerId,
            InvoiceDate,
            STRFTIME("%W", InvoiceDate),
            STRFTIME("%M", InvoiceDate),
            STRFTIME("%Y", InvoiceDate),
            BillingAddress,
            BillingCity,
            BillingState,
            BillingCountry,
            BillingPostalCode,
            Total
        FROM invoices
    """

executer_requete(creation_v_invoices, conn)

In [17]:

v_invoices = pd.read_sql_query("SELECT * FROM v_invoices;", conn)
v_invoices

,InvoiceId,CustomerId,InvoiceDate,InvoiceSemaine,InvoiceMois,InvoiceAnnée,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,00,00,2009,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,00,00,2009,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,00,00,2009,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,01,00,2009,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,01,00,2009,69 Salem Street,Boston,MA,USA,2113,13.86
...,...,...,...,...,...,...,...,...,...,...,...,...
407,408,25,2013-12-05 00:00:00,48,00,2013,319 N. Frances Street,Madison,WI,USA,53703,3.96
408,409,29,2013-12-06 00:00:00,48,00,2013,796 Dundas Street West,Toronto,ON,Canada,M6J 1V1,5.94
409,410,35,2013-12-09 00:00:00,49,00,2013,"Rua dos Campeões Europeus de Viena, 4350",Porto,None,Portugal,None,8.91
410,411,44,2013-12-14 00:00:00,49,00,2013,Porthaninkatu 9,Helsinki,None,Finland,00530,13.86


B. Exploiter la vue v_invoices ainsi crée pour déterminer les ventes annuelles par pays.

In [18]:
query = """
SELECT BillingCountry AS pays,
SUM(Total) AS "Total Ventes par année",
InvoiceAnnée AS Année
FROM v_invoices
GROUP BY InvoiceAnnée, pays
"""
results = pd.read_sql_query(query, conn)
results

,pays,Total Ventes par année,Année
0,Australia,11.88,2009
1,Austria,1.98,2009
2,Belgium,6.93,2009
3,Brazil,37.62,2009
4,Canada,57.42,2009
...,...,...,...
96,Poland,0.99,2013
97,Portugal,24.75,2013
98,Spain,11.88,2013
99,USA,85.14,2013


### Vue 4: A vous de jouer
Créer et exploiter une nouvelle vue de votre choix  

Nouvelle vue v_genres des tous les genres de musique reprenant son genre, le nombre de pistes, la durée de toutes les pistes (en minutes), le coût total et le coût moyen

In [19]:
creation_vue_genre = """CREATE VIEW v_genres (
        Genre,
        "Nb pistes",
        "Durée (min)",
        "côut total",
        "côut moyens")
     AS
        SELECT genres.Name AS Genre,
        COUNT(tracks.TrackId) AS "Nb pistes",
        SUM(tracks.Milliseconds)/60000 AS "Durée (min)",
        SUM(tracks.UnitPrice) AS "côut total",
        AVG(tracks.UnitPrice) AS "côut moyens"
        FROM genres 
        LEFT JOIN tracks ON genres.GenreId=tracks.GenreId 
        GROUP BY genres.GenreId
        ORDER BY Genres.Name
        """

In [20]:
executer_requete(creation_vue_genre, conn)

Erreur lors de l'execution de la requête
table v_genres already exists


In [21]:
v_genres = pd.read_sql_query("SELECT * FROM v_genres;", conn)
v_genres

,Genre,Nb pistes,Durée (min),côut total,côut moyens
0,Alternative,40,176,39.60,0.99
1,Alternative & Punk,332,1296,328.68,0.99
2,Blues,81,364,80.19,0.99
3,Bossa Nova,15,54,14.85,0.99
4,Classical,74,362,73.26,0.99
5,Comedy,17,449,33.83,1.99
6,Drama,64,2746,127.36,1.99
7,Easy Listening,24,75,23.76,0.99
8,Electronica/Dance,30,151,29.70,0.99
9,Heavy Metal,28,138,27.72,0.99
